In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 16, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv6 = nn.Conv2d(16, 32, 3)

        # Replace conv7 with a 1x1 kernel instead of a 3x3 kernel
        self.conv7 = nn.Conv2d(32, 10, 1)  # 1x1 kernel

        # Define dropout with 10% probability
        self.dropout = nn.Dropout(p=0.15)  # 15% dropout rate

        # Define Batch Normalization layers
        self.bn1 = nn.BatchNorm2d(16)  # BatchNorm for the output of conv1
        self.bn2 = nn.BatchNorm2d(16)  # BatchNorm for the output of conv2
        self.bn3 = nn.BatchNorm2d(32)  # BatchNorm for the output of conv3
        self.bn4 = nn.BatchNorm2d(16)  # BatchNorm for the output of conv4
        self.bn5 = nn.BatchNorm2d(16) # BatchNorm for the output of conv5
        self.bn6 = nn.BatchNorm2d(32) # BatchNorm for the output of conv6

        # Global Average Pooling layer
        self.gap = nn.AdaptiveAvgPool2d(1)  # Reduces each feature map to 1x1

    def forward(self, x):
        # First convolution block with BatchNorm, ReLU, and Dropout
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))  # Conv1 -> BN -> ReLU -> Conv2 -> BN -> ReLU -> Pool1
        x = self.dropout(x)  # Apply dropout after activation

        # Second convolution block with BatchNorm, ReLU, and Dropout
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))  # Conv3 -> BN -> ReLU -> Conv4 -> BN -> ReLU -> Pool2
        x = self.dropout(x)  # Apply dropout after activation

        # Third convolution block with BatchNorm, ReLU, and Dropout
        x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))  # Conv5 -> BN -> ReLU -> Conv6 -> BN -> ReLU
        x = self.dropout(x)  # Apply dropout after activation

        # Apply Global Average Pooling (GAP) to reduce each feature map to 1x1
        x = self.gap(x)  # Now the size of x is (batch_size, 128, 1, 1)

        # Flatten the output to feed into the final fully connected layer
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128)

        # Final convolution layer (now with 1x1 kernel)
        # x = self.conv7(x.unsqueeze(-1).unsqueeze(-1))  # Convert the flattened vector into a form for conv7 (needs to be (batch_size, 128, 1, 1))

        # Return log softmax for classification output
        return F.log_softmax(x, dim=1)  # Log softmax for classification output


In [3]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9           [-1, 16, 14, 14]           4,624
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
          Dropout-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 16, 7, 7]           2,320
      BatchNorm2d-14             [-1, 1

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
# torch.manual_seed(1)
# batch_size = 128

# train_transform = transforms.Compose([
#     transforms.RandomRotation(20),                # Randomly rotate images by up to 20 degrees
#     transforms.RandomAffine(0,                    # Randomly apply affine transformations
#                            translate=(0.2, 0.2),    # Randomly shift images by up to 20%
#                            shear=20),            # Fill black for empty areas
#     transforms.RandomResizedCrop(28, scale=(0.8, 1.2)), # Random zoom/crop
#     transforms.ToTensor(),                        # Convert to tensor
#     transforms.Normalize((0.1307,), (0.3081,))     # Normalize
# ])

# # Define training and testing datasets with transformations
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=train_transform),
#     batch_size=batch_size, shuffle=True, **kwargs
# )

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=batch_size, shuffle=False, **kwargs
# )

In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("EPOCH: ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    if accuracy >= 99.4:
      break

EPOCH:  1


loss=0.10297595709562302 batch_id=468: 100%|██████████| 469/469 [01:40<00:00,  4.65it/s]



Test set: Average loss: 0.1136, Accuracy: 9781/10000 (97.8100%)

EPOCH:  2


loss=0.11357773095369339 batch_id=468: 100%|██████████| 469/469 [01:38<00:00,  4.75it/s]



Test set: Average loss: 0.2572, Accuracy: 9242/10000 (92.4200%)

EPOCH:  3


...


Test set: Average loss: 0.0284, Accuracy: 9913/10000 (99.1300%)

EPOCH:  12


loss=0.026190033182501793 batch_id=468: 100%|██████████| 469/469 [01:40<00:00,  4.66it/s]



Test set: Average loss: 0.0252, Accuracy: 9934/10000 (99.3400%)

EPOCH:  13


loss=0.03852536156773567 batch_id=468: 100%|██████████| 469/469 [01:39<00:00,  4.71it/s]



Test set: Average loss: 0.0206, Accuracy: 9941/10000 (99.4100%)



In [7]:
torch.save(model.state_dict(), 'model.pth')